In [107]:
from collections import defaultdict, Counter
import re

# Byte Pair Encoding (BPE)

In [108]:
# Define the example corpus
corpus = "low low low low low lower lower newest newest newest newest newest newest widest widest widest"
example_sentence = "lowering the newest wide"

In [109]:
def get_subword_pairs(vocab) -> dict:
    """Get frequency of adjacent subword pairs in the vocabulary."""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs


def merge_vocab(pair, v_in) -> dict:
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out


def byte_pair_encoding(corpus, num_merges) -> tuple[dict, list]:
    # Initialize vocabulary with individual characters
    vocab = {' '.join(word) + ' </w>': count for word, count in Counter(corpus.split()).items()}
    final_vocab = []

    for i in range(num_merges):
        pairs = get_subword_pairs(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)
        final_vocab.append(best[0] + best[1])
        print(f"Iteration {i + 1}: Merged {best}")

    return vocab, final_vocab


def tokenize_word(word, start_char_index):
    start_char = word[start_char_index]
    end_char_index = None
    chars = ""
    token = None
    index = start_char_index
    for char in word[start_char_index:]:
        index += 1
        chars += char
        if chars in final_bpe_vocab or chars == "</w>":
            token = chars
            end_char_index = index
    if token is None:
        return start_char, start_char_index+1
    else:
        return token, end_char_index


def apply_bpe_tokenizer(sequence) -> list:
    encoded = []
    words = [word + "</w>" for word in sequence.split()]

    for word in words:
        word_char_index = 0
        while word_char_index is not len(word):
            token, word_char_index = tokenize_word(word, word_char_index)
            encoded.append(token)

    return encoded


num_merges = 5
vocab, final_bpe_vocab = byte_pair_encoding(corpus, num_merges)
print("\nFinal vocabulary:", final_bpe_vocab)
print("Merged vocabulary:", vocab)

# Apply BPE tokenization
tokenized = apply_bpe_tokenizer(example_sentence)
print(f"\nTokenization of '{example_sentence}': {tokenized}")

Iteration 1: Merged ('e', 's')
Iteration 2: Merged ('es', 't')
Iteration 3: Merged ('est', '</w>')
Iteration 4: Merged ('l', 'o')
Iteration 5: Merged ('lo', 'w')

Final vocabulary: ['es', 'est', 'est</w>', 'lo', 'low']
Merged vocabulary: {'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}

Tokenization of 'lowering the newest wide': ['low', 'e', 'r', 'i', 'n', 'g', '</w>', 't', 'h', 'e', '</w>', 'n', 'e', 'w', 'est</w>', 'w', 'i', 'd', 'e', '</w>']


# Wordpiece Tokenization

In [110]:
def get_subword_pairs(vocab) -> dict:
    """Get frequency of adjacent subword pairs in the vocabulary."""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq
    return pairs


def merge_vocab(pair, v_in) -> dict:
    """Merge the most frequent pair in the vocabulary."""
    v_out = {}
    bigram = re.escape(" ".join(pair))
    p = re.compile(r"(?<!\S)" + bigram + r"(?!\S)")  # Match full token pairs

    for word in v_in:
        # Merge the pair into a single token
        w_out = p.sub("".join(pair), word)
        v_out[w_out] = v_in[word]

    return v_out


def compute_score(pair, pairs, vocab) -> float:
    """Compute the WordPiece score: P(xy) / (P(x) * P(y))"""
    xy = pairs[pair]  # Frequency of merged token
    x = sum(vocab[word] for word in vocab if pair[0] in word.split())
    y = sum(vocab[word] for word in vocab if pair[1] in word.split())
    return xy / (x * y) if x * y > 0 else 0  # Avoid division by zero


def wordpiece_tokenization(corpus, vocab_size) -> tuple[dict, set]:
    """Train WordPiece tokenization."""
    # Step 1: Initialize vocabulary with characters + [UNK]
    word_freqs = Counter(corpus.split())
    vocab = {" ".join(word) + " </w>": freq for word, freq in word_freqs.items()}
    final_vocab = set(char for word in word_freqs for char in word) | {"[UNK]"}

    while len(final_vocab) < vocab_size:
        pairs = get_subword_pairs(vocab)
        if not pairs:
            break

        # Step 2: Select merge based on WordPiece scoring function
        best_pair = max(pairs, key=lambda p: compute_score(p, pairs, vocab))
        merged_token = best_pair[0] + best_pair[1]

        # Add to vocabulary (use ## if not at word start)
        if best_pair[0] in final_vocab:
            merged_token = "##" + merged_token if not best_pair[0].endswith("</w>") else merged_token
        final_vocab.add(merged_token)

        # Step 3: Merge the vocabulary
        vocab = merge_vocab(best_pair, vocab)
        print(f"Merged: {best_pair} -> {merged_token}")

    return vocab, final_vocab


def apply_wordpiece_tokenizer(sentence) -> list:
    return "none"


# Example Usage
vocab_size = 20
vocab, final_wp_vocab = wordpiece_tokenization(corpus, vocab_size)

print("\nFinal vocabulary:", final_wp_vocab)
print("Merged vocabulary:", vocab)

# Apply BPE tokenization
tokenized = apply_wordpiece_tokenizer(example_sentence)
print(f"\nTokenization of '{example_sentence}': {tokenized}")

Merged: ('i', 'd') -> ##id
Merged: ('l', 'o') -> ##lo
Merged: ('s', 't') -> ##st
Merged: ('e', 'r') -> ##er
Merged: ('n', 'e') -> ##ne
Merged: ('e', 'st') -> ##est
Merged: ('id', 'est') -> idest
Merged: ('lo', 'w') -> low
Merged: ('low', 'er') -> ##lower

Final vocabulary: {'##est', 'low', 's', 'n', 't', 'o', '##lo', 'idest', '##er', 'r', 'i', '##ne', 'e', '[UNK]', '##lower', '##st', '##id', 'd', 'w', 'l'}
Merged vocabulary: {'low </w>': 5, 'lower </w>': 2, 'ne w est </w>': 6, 'w idest </w>': 3}

Tokenization of 'lowering the newest wide': none


# Other implemplementations

### BPE

In [111]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialize BPE tokenizer
bpw_tokenizer = Tokenizer(BPE())

# Use whitespace to split words
bpw_tokenizer.pre_tokenizer = Whitespace()

# Train on a corpus
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
bpw_tokenizer.train_from_iterator([corpus], trainer=trainer)

# Show the final vocab
vocab = bpw_tokenizer.get_vocab()
print("Final vocab:", vocab)

# Tokenize a sentence
tokenized = bpw_tokenizer.encode(example_sentence)
print(f"\nTokenization of '{example_sentence}': {tokenized.tokens}")




Final vocab: {'[PAD]': 3, 'est': 16, 'es': 15, 'dest': 22, '[UNK]': 0, 'low': 18, 'l': 8, 'o': 10, 'n': 9, 'new': 20, 'idest': 23, 'i': 7, 'widest': 24, 'lo': 17, 'r': 11, 'd': 5, 'er': 25, '[MASK]': 4, 'ew': 19, 'w': 14, 't': 13, 'e': 6, '[SEP]': 2, 'newest': 21, 's': 12, '[CLS]': 1, 'lower': 26}

Tokenization of 'lowering the newest wide': ['lower', 'i', 'n', 't', 'e', 'newest', 'w', 'i', 'd', 'e']


### WordPiece

In [112]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer

# Initialize WordPiece tokenizer
wp_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

wp_tokenizer.pre_tokenizer = Whitespace()

# Train on a corpus
trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
wp_tokenizer.train_from_iterator([corpus], trainer=trainer)

# Show the final vocab
vocab = wp_tokenizer.get_vocab()
print("Final vocab:", vocab)

# Tokenize a sentence
tokenized = wp_tokenizer.encode(example_sentence)
print(f"\nTokenization of '{example_sentence}': {tokenized.tokens}")




Final vocab: {'wi': 30, '[MASK]': 4, '[CLS]': 1, 's': 12, 'w': 14, 't': 13, '##est': 24, 'i': 7, 'l': 8, '[UNK]': 0, 'widest': 32, '##er': 33, '##i': 20, 'd': 5, '##d': 21, '##o': 15, 'e': 6, '##r': 22, '##dest': 31, 'low': 26, '##w': 16, '##es': 23, 'lower': 34, '##west': 28, 'r': 11, 'newest': 29, '##t': 19, 'lo': 25, '[PAD]': 3, 'o': 10, '##e': 17, 'n': 9, '##s': 18, '[SEP]': 2, 'ne': 27}

Tokenization of 'lowering the newest wide': ['[UNK]', '[UNK]', 'newest', 'wi', '##d', '##e']


# Hugging Face Implementations
BertTokenizer, RobertaTokenizer

In [113]:
from transformers import BertTokenizer, RobertaTokenizer

# Load pre-trained tokenizers
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize using BERT tokenizer
bert_tokens = bert_tokenizer.tokenize(example_sentence)

# Tokenize using RoBERTa tokenizer
roberta_tokens = roberta_tokenizer.tokenize(example_sentence)

print("BERT Tokens:", bert_tokens)
print("RoBERTa Tokens:", roberta_tokens)

BERT Tokens: ['lowering', 'the', 'newest', 'wide']
RoBERTa Tokens: ['lower', 'ing', 'Ġthe', 'Ġnewest', 'Ġwide']


# Compare the tokenization of the same sentence using different tokenizers

In [114]:
print(f"Original Sentence: {example_sentence}")

custom_bpe_tokens = apply_bpe_tokenizer(example_sentence)
custom_wordpiece_tokens = apply_wordpiece_tokenizer(example_sentence)

lib_bpe_tokenized = bpw_tokenizer.encode(example_sentence)
lib_wp_tokenized = wp_tokenizer.encode(example_sentence)

bert_tokens = bert_tokenizer.tokenize(example_sentence)
roberta_tokens = roberta_tokenizer.tokenize(example_sentence)

print("\nCustom BPE Tokens:", custom_bpe_tokens)
print("Library BPE Tokens:", lib_bpe_tokenized.tokens)

print("\nCustom WordPiece Tokens:", custom_wordpiece_tokens)
print("Library WordPiece Tokens:", lib_wp_tokenized.tokens)
print("\nBERT Tokens:", bert_tokens)
print("RoBERTa Tokens:", roberta_tokens)

Original Sentence: lowering the newest wide

Custom BPE Tokens: ['low', 'e', 'r', 'i', 'n', 'g', '</w>', 't', 'h', 'e', '</w>', 'n', 'e', 'w', 'est</w>', 'w', 'i', 'd', 'e', '</w>']
Library BPE Tokens: ['lower', 'i', 'n', 't', 'e', 'newest', 'w', 'i', 'd', 'e']

Custom WordPiece Tokens: none
Library WordPiece Tokens: ['[UNK]', '[UNK]', 'newest', 'wi', '##d', '##e']

BERT Tokens: ['lowering', 'the', 'newest', 'wide']
RoBERTa Tokens: ['lower', 'ing', 'Ġthe', 'Ġnewest', 'Ġwide']
